In [ ]:
import MINST_Train
import MIPVerify
import Ours
import numpy as np
import csv

MNIST_DIR = "../Data/MINST_Train"
TRAIN_DATA = "train-images.idx3-ubyte"
TRAIN_LABEL = "train-labels.idx1-ubyte"
TEST_DATA = "t10k-images.idx3-ubyte"
TEST_LABEL = "t10k-labels.idx1-ubyte"
h1, h2, e = 32, 16, 1

In [ ]:
hidden1_accuracy = []
hidden2_accuracy = []
for index in range(10):
    a, b = Ours.nodeImportanceCompute(index, MNIST_DIR, 
                                 TRAIN_DATA, TRAIN_LABEL,
                                 TEST_DATA, TEST_LABEL)
    hidden1_accuracy.append(a)    
    hidden2_accuracy.append(b)
    print("Compute importance of index ", index, " done!")
    print(hidden1_accuracy)

In [ ]:
with open('hidden1_accuracy.csv', 'w', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    for row in hidden1_accuracy:
        writer.writerow(row)

with open('hidden2_accuracy.csv', 'w', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    for row in hidden2_accuracy:
        writer.writerow(row)

In [ ]:
# importance lable: 0(不重要), 1(一般重要), 2(较重要)
initial_accuracy = []
for index in range(10):
    mlp = MINST_Train.MNIST_MLP(hidden1=h1, hidden2=h2, max_epoch=e, data_dir=MNIST_DIR, train_data=TRAIN_DATA,
                                    train_label=TRAIN_LABEL, test_data=TEST_DATA, test_label=TEST_LABEL)
    mlp.load_data()
    mlp.build_model()
    mlp.init_model()
    mlp.load_model('../Data/mlp-%d-%d-%depoch.npy' % (h1, h2, e))
    initial_accuracy.append(mlp.evaluateByLabel()[index])
tmp1 = []
tmp2 = []
for index, each in enumerate(hidden1_accuracy):
    tmp = []
    for value in each:
        if value - initial_accuracy[index] >= 0:
            tmp.append(0)
        elif value - initial_accuracy[index] >= -0.01:
            tmp.append(1)
        else:
            tmp.append(2)
    tmp1.append(tmp)
for index, each in enumerate(hidden2_accuracy):
    tmp = []
    for value in each:
        if value - initial_accuracy[index] >= 0:
            tmp.append(0)
        elif value - initial_accuracy[index] >= -0.01:
            tmp.append(1)
        else:
            tmp.append(2)
    tmp2.append(tmp)
hidden1_accuracy = tmp1
hidden2_accuracy = tmp2

In [ ]:
datas = []
for i in range(1, 101):
    tmp = []
    tmp.append(i)
    print(i)
    for j in range(10):
        mlp = MINST_Train.MNIST_MLP(hidden1=h1, hidden2=h2, max_epoch=e, 
                                    data_dir=MNIST_DIR, train_data=TRAIN_DATA,
                                    train_label=TRAIN_LABEL, 
                                    test_data=TEST_DATA, test_label=TEST_LABEL)

        # 选取一张图片
        index = i  # 图片序号
        epsilon = 25  # 扰动范围
        target_index = j

        # 创建MIP模型,创建变量并更新变量空间
        model = Ours.Ours(nnmodel=mlp, index=index, epsilon=epsilon, 
                         target_index=target_index)
        model.load_model(h1, h2, e)
        
        if target_index == model.label:
            tmp.append('none')
            tmp.append('none')
            continue
        
        input_with_PA, perturbation_added = model.addPreconditionConstr()

        bound = np.zeros((mlp.input_size, 2))
        bound[:, 0] = mlp.test_data[index][0:-1] - epsilon
        bound[:, 1] = mlp.test_data[index][0:-1] + epsilon
        np.minimum(bound, 255)
        np.maximum(bound, 0)
        hidden1, bound, temp1 = model.addNNConstr(input_with_PA, mlp.hidden1, 
                                                  mlp.fc1.weight, mlp.fc1.bias, 
                                                  bound, hidden1_accuracy[target_index])
        hidden2, bound, temp2 = model.addNNConstr(hidden1, mlp.hidden2, mlp.fc2.weight, 
                                                  mlp.fc2.bias, bound, hidden2_accuracy[target_index])
        model.addPostconditionConstr(hidden2, mlp.out_classes, mlp.fc3.weight, mlp.fc3.bias)
        model.setObj(perturbation_added)

        # 执行最优化过程
        model.mipmodel.optimize()
        if model.mipmodel.status == 3:
            tmp.append('infeasible')
            tmp.append('infeasible')
            continue
        tmp.append(model.mipmodel.objVal)
        tmp.append(model.mipmodel.Runtime)
    datas.append(tmp)
#         model.mipmodel.write("model1.lp")
#         print("Obj:", model.mipmodel.objVal)
#         print("Time:", model.mipmodel.Runtime)

In [ ]:
with open('result_ours.csv', 'a', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    for row in datas:
        writer.writerow(row)

In [ ]:
datas = []
for i in range(1, 101):
    tmp = []
    tmp.append(i)
    print(i)
    for j in range(10):
        mlp = MINST_Train.MNIST_MLP(hidden1=h1, hidden2=h2, max_epoch=e, data_dir=MNIST_DIR, train_data=TRAIN_DATA, train_label=TRAIN_LABEL, test_data=TEST_DATA, test_label=TEST_LABEL)

        # 选取一张图片
        index = i   # 图片序号
        epsilon = 25   # 扰动范围
        # picture = mlp.test_data[index][0:-1]
        # label = mlp.test_data[index][-1]    # label = 7
        target_index = j
        # print(label)

        # 创建MIP模型,创建变量并更新变量空间
        model = MIPVerify.MIPVerify(nnmodel=mlp, index=index, epsilon=epsilon, target_index=target_index)
        model.load_model(h1, h2, e)
        input_with_PA, perturbation_added = model.addPreconditionConstr()

        bound = np.zeros((mlp.input_size, 2))
        bound[:, 0] = mlp.test_data[index][0:-1] - epsilon
        bound[:, 1] = mlp.test_data[index][0:-1] + epsilon
        np.minimum(bound, 255)
        np.maximum(bound, 0)
        hidden1, bound, temp1 = model.addNNConstr(input_with_PA, mlp.hidden1, mlp.fc1.weight, mlp.fc1.bias, bound)
        hidden2, bound, temp2 = model.addNNConstr(hidden1, mlp.hidden2, mlp.fc2.weight, mlp.fc2.bias, bound)
        model.addPostconditionConstr(hidden2, mlp.out_classes, mlp.fc3.weight, mlp.fc3.bias)
        model.setObj(perturbation_added)

        # 执行最优化过程
        model.mipmodel.optimize()
        if model.mipmodel.status == 3:
            tmp.append('infeasible')
            tmp.append('infeasible')
            continue
        tmp.append(model.mipmodel.objVal)
        tmp.append(model.mipmodel.Runtime)
    datas.append(tmp)
#         model.mipmodel.write("model1.lp")
#         print("Obj:", model.mipmodel.objVal)
#         print("Time:", model.mipmodel.Runtime)

In [ ]:
with open('result_MIPVerify_25.csv', 'a', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    writer.writerow([ ])
    for row in datas:
        writer.writerow(row)

In [ ]:
datas = []
for i in range(1, 101):
    tmp = []
    tmp.append(i)
    print(i)
    for j in range(10):
        mlp = MINST_Train.MNIST_MLP(hidden1=h1, hidden2=h2, max_epoch=e, data_dir=MNIST_DIR, train_data=TRAIN_DATA, train_label=TRAIN_LABEL, test_data=TEST_DATA, test_label=TEST_LABEL)

        # 选取一张图片
        index = i   # 图片序号
        epsilon = 50   # 扰动范围
        # picture = mlp.test_data[index][0:-1]
        # label = mlp.test_data[index][-1]    # label = 7
        target_index = j
        # print(label)

        # 创建MIP模型,创建变量并更新变量空间
        model = MIPVerify.MIPVerify(nnmodel=mlp, index=index, epsilon=epsilon, target_index=target_index)
        model.load_model(h1, h2, e)
        input_with_PA, perturbation_added = model.addPreconditionConstr()

        bound = np.zeros((mlp.input_size, 2))
        bound[:, 0] = mlp.test_data[index][0:-1] - epsilon
        bound[:, 1] = mlp.test_data[index][0:-1] + epsilon
        np.minimum(bound, 255)
        np.maximum(bound, 0)
        hidden1, bound, temp1 = model.addNNConstr(input_with_PA, mlp.hidden1, mlp.fc1.weight, mlp.fc1.bias, bound)
        hidden2, bound, temp2 = model.addNNConstr(hidden1, mlp.hidden2, mlp.fc2.weight, mlp.fc2.bias, bound)
        model.addPostconditionConstr(hidden2, mlp.out_classes, mlp.fc3.weight, mlp.fc3.bias)
        model.setObj(perturbation_added)

        # 执行最优化过程
        model.mipmodel.optimize()
        if model.mipmodel.status == 3:
            tmp.append('infeasible')
            tmp.append('infeasible')
            continue
        tmp.append(model.mipmodel.objVal)
        tmp.append(model.mipmodel.Runtime)
    datas.append(tmp)
#         model.mipmodel.write("model1.lp")
#         print("Obj:", model.mipmodel.objVal)
#         print("Time:", model.mipmodel.Runtime)

In [ ]:
with open('result_MIPVerify_50.csv', 'a', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    writer.writerow([ ])
    for row in datas:
        writer.writerow(row)

In [ ]:
datas = []

for i in rnge(1, 101):
    tmp = []
    tmp.append(i)
    print(i)
    for j in range(10):
        mlp = MINST_Train.MNIST_MLP(hidden1=h1, hidden2=h2, max_epoch=e, 
                                    data_dir=MNIST_DIR, train_data=TRAIN_DATA,
                                    train_label=TRAIN_LABEL, 
                                    test_data=TEST_DATA, test_label=TEST_LABEL)

        # 选取一张图片
        index = i  # 图片序号
        epsilon = 25  # 扰动范围
        target_index = j

        # 创建MIP模型,创建变量并更新变量空间
        model = Ours.Ours(nnmodel=mlp, index=index, epsilon=epsilon, 
                         target_index=target_index)
        model.load_model(h1, h2, e)
        
        if target_index == model.label:
            tmp.append('none')
            tmp.append('none')
            continue
        
        input_with_PA, perturbation_added = model.addPreconditionConstr()

        bound = np.zeros((mlp.input_size, 2))
        bound[:, 0] = mlp.test_data[index][0:-1] - epsilon
        bound[:, 1] = mlp.test_data[index][0:-1] + epsilon
        np.minimum(bound, 255)
        np.maximum(bound, 0)
        hidden1, bound, temp1 = model.addNNConstr(input_with_PA, mlp.hidden1, 
                                                  mlp.fc1.weight, mlp.fc1.bias, 
                                                  bound, [0 for i in range(32)])
        hidden2, bound, temp2 = model.addNNConstr(hidden1, mlp.hidden2, mlp.fc2.weight, 
                                                  mlp.fc2.bias, bound, [0 for i in range(16)])
        model.addPostconditionConstr(hidden2, mlp.out_classes, mlp.fc3.weight, mlp.fc3.bias)
        model.setObj(perturbation_added)

        # 执行最优化过程
        model.mipmodel.optimize()
        if model.mipmodel.status == 3:
            tmp.append('infeasible')
            tmp.append('infeasible')
            continue
        tmp.append(model.mipmodel.objVal)
        tmp.append(model.mipmodel.Runtime)
    datas.append(tmp)
#         model.mipmodel.write("model1.lp")
#         print("Obj:", model.mipmodel.objVal)
#         print("Time:", model.mipmodel.Runtime)

In [ ]:
with open('result_LinearRelaxation.csv', 'a', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    writer.writerow([ ])
    for row in datas:
        writer.writerow(row)

In [ ]:
datas = []

for i in range(1, 101):
    tmp = []
    tmp.append(i)
    print(i)
    for j in range(10):
        mlp = MINST_Train.MNIST_MLP(hidden1=h1, hidden2=h2, max_epoch=e, 
                                    data_dir=MNIST_DIR, train_data=TRAIN_DATA,
                                    train_label=TRAIN_LABEL, 
                                    test_data=TEST_DATA, test_label=TEST_LABEL)

        # 选取一张图片
        index = i  # 图片序号
        epsilon = 25  # 扰动范围
        target_index = j

        # 创建MIP模型,创建变量并更新变量空间
        model = Ours.Ours(nnmodel=mlp, index=index, epsilon=epsilon, 
                         target_index=target_index)
        model.load_model(h1, h2, e)
        
        if target_index == model.label:
            tmp.append('none')
            tmp.append('none')
            continue
        
        input_with_PA, perturbation_added = model.addPreconditionConstr()

        bound = np.zeros((mlp.input_size, 2))
        bound[:, 0] = mlp.test_data[index][0:-1] - epsilon
        bound[:, 1] = mlp.test_data[index][0:-1] + epsilon
        np.minimum(bound, 255)
        np.maximum(bound, 0)
        hidden1, bound, temp1 = model.addNNConstr(input_with_PA, mlp.hidden1, 
                                                  mlp.fc1.weight, mlp.fc1.bias, 
                                                  bound, [1 for i in range(32)])
        hidden2, bound, temp2 = model.addNNConstr(hidden1, mlp.hidden2, mlp.fc2.weight, 
                                                  mlp.fc2.bias, bound, [1 for i in range(16)])
        model.addPostconditionConstr(hidden2, mlp.out_classes, mlp.fc3.weight, mlp.fc3.bias)
        model.setObj(perturbation_added)

        # 执行最优化过程
        model.mipmodel.optimize()
        if model.mipmodel.status == 3:
            tmp.append('infeasible')
            tmp.append('infeasible')
            continue
        tmp.append(model.mipmodel.objVal)
        tmp.append(model.mipmodel.Runtime)
    datas.append(tmp)
#         model.mipmodel.write("model1.lp")
#         print("Obj:", model.mipmodel.objVal)
#         print("Time:", model.mipmodel.Runtime)

In [ ]:
with open('result_TriangleRelaxation.csv', 'a', newline='') as csvfile:
    writer  = csv.writer(csvfile)
    writer.writerow([ ])
    for row in datas:
        writer.writerow(row)